In [8]:
import time
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from QFTAddition import add
from qiskit import execute
from qiskit import Aer
from BitwiseOperations import rshift
from QFTSubtraction import sub

# modmult = x * y mod m
# Inputs: x,y,m
x = input('Enter the binary number for x: ')
y = input('Enter the binary number for y: ')
m = input('Enter the binary number for m: ')

# x = '110101'
# y = '000001'
# m = '100011'

start_time = time.time()

# Measuring the length of x
n = len(x)

# Assigning Quantum registers for Quantum operation
x_reg = QuantumRegister(n+1)
y_reg = QuantumRegister(n+2)
y_reg_0 = QuantumRegister(1)
m_reg = QuantumRegister(n+2)
a_reg = QuantumRegister(n+2)
u_reg = QuantumRegister(n+1)
onecubit = QuantumRegister(1)

# Assigning classical registers to store our results from Quantum registers
a_cl_reg = ClassicalRegister(n+2)
u_cl_reg = ClassicalRegister(n+1)
cl_reg = ClassicalRegister(n+1)
one_cl_reg = ClassicalRegister(1)


# creating seperate Quantum circuits for different operations to speedup the process
circ_u = QuantumCircuit(u_reg, y_reg_0, u_cl_reg)
circ_a = QuantumCircuit(a_reg, y_reg, m_reg, a_cl_reg, onecubit, one_cl_reg)

y0 = y[n - 1];
if y0 == '1':
    circ_u.x(y_reg_0) 

# Loading inputs to Quantum registers
for i in range(n):
    if y[i] == '1':
        circ_a.x(y_reg[n - i - 1])
for i in range(n):
    if m[i] == '1':
        circ_a.x(m_reg[n - i - 1])
        
        
for i in range(n):
    if x[n - i - 1] == '1':            
        add(u_reg, y_reg_0, circ_u)
    circ_u.measure(u_reg[0], u_cl_reg[0])  
    result = execute(circ_u,backend=Aer.get_backend('qasm_simulator'), shots=1).result().get_counts(circ_u)
    measure_u = int((list(result.keys())[0]))
#     print('measure_u: ', measure_u)
    
    if x[n - i - 1] == '1':
        add(a_reg, y_reg, circ_a)
    if measure_u == 1:
        add(a_reg, m_reg, circ_a)
    rshift(circ_a, a_reg, n + 2, onecubit)
    circ_a.measure(a_reg, a_cl_reg)
    circ_a.measure(onecubit, one_cl_reg)
    result = execute(circ_a,backend=Aer.get_backend('qasm_simulator'), shots=1).result().get_counts(circ_a)
    total = list(result.keys())[0]    
    measure_a = total[2:]
#     print(measure_a)
    
    measure_onecubit = int(total[0])
    if measure_onecubit == 1:
        circ_a.x(onecubit)    
    
    # loading a0 to u0
    if measure_a[n + 1] == '1':
        if measure_u == 0:
            circ_u.x(u_reg[0])
    else:
        if measure_u == 1:
            circ_u.x(u_reg[0])


if (int(measure_a) >= int(m)):
    sub(a_reg, m_reg, circ_a)
    
circ_a.measure(a_reg, a_cl_reg)
result = execute(circ_a,backend=Aer.get_backend('qasm_simulator'), shots=1).result().get_counts(circ_a)
total = list(result.keys())[0]    
final_a = total[2:]
print("Result for Montgomery modular multiplication: ",final_a)

print("Time taken to execute: %s seconds" % (time.time() - start_time))

Enter the binary number for x: 110
Enter the binary number for y: 110
Enter the binary number for m: 111
Result for Montgomery modular multiplication:  00001
Time taken to execute: 0.7048022747039795 seconds


In [ ]:
help(result.keys)